# CRF+LSTM

requirement:

keras 2.2.4

tensorflow 1.13

pip install git+https://www.github.com/keras-team/keras-contrib.git

In [67]:
import re
import os
import pandas as pd

In [68]:
char_vocab_path = "CRF/data/char_vocabs.txt" # 字典文件
special_words = ['<PAD>', '<UNK>'] # 特殊词表示
label2idx = {'O': 0,
             'B-DISEASE': 1, 'B-DISEASE_GROUP': 2,
             'B-DRUG_DOSAGE': 3, 'B-DRUG_EFFICACY': 4,
             'B-DRUG_INGREDIENT': 5, 'B-DRUG_TASTE': 6,
             'B-FOOD_GROUP':7, 'B-PERSON_GROUP':8,
             'B-SYMPTOM':9, 'B-SYNDROME':10,
             'I-DISEASE': 11, 'I-DISEASE_GROUP': 12,
             'I-DRUG_DOSAGE': 13, 'I-DRUG_EFFICACY': 14,
             'I-DRUG_INGREDIENT': 15, 'I-DRUG_TASTE': 16,
             'I-FOOD_GROUP':17, 'I-PERSON_GROUP':18,
             'I-SYMPTOM':19, 'I-SYNDROME':20
            }

# 索引和BIO标签对应
idx2label = {idx: label for label, idx in label2idx.items()}

# 读取字符词典文件
with open(char_vocab_path, "r", encoding="utf8") as fo:
    char_vocabs = [line.strip() for line in fo]
char_vocabs = special_words + char_vocabs

# 字符和索引编号对应
idx2vocab = {idx: char for idx, char in enumerate(char_vocabs)}
vocab2idx = {char: idx for idx, char in idx2vocab.items()}

In [69]:
# 读取训练语料
def read_corpus(corpus_path, vocab2idx, label2idx):
    with open(corpus_path, encoding='utf-8') as fr:
        lines = fr.readlines()
    sent_, tag_ = [], []
    for letter in lines:
        [char,label,_] = re.split('\t|\n',letter)
        char = re.sub(' |\*|<|>','_',char)
        sent_.append(char)
        tag_.append(label)
    sent_ids = [vocab2idx[char] if char in vocab2idx else vocab2idx['<UNK>'] for char in sent_]
    tag_ids = [label2idx[label] if label in label2idx else 0 for label in tag_]
    return sent_ids, tag_ids

In [70]:
train_datas = []
train_labels = []
files = os.listdir('data/train_data')
for file in files:
    train_data_path_i = 'data/train_data/'+file
    train_datas_i, train_labels_i = read_corpus(train_data_path_i, vocab2idx, label2idx)
    train_datas.append(train_datas_i)
    train_labels.append(train_labels_i)

In [71]:
valid_datas = []
valid_labels = []
files = os.listdir('data/valid_data')
for file in files:
    valid_data_path_i = 'data/valid_data/'+file
    valid_datas_i, valid_labels_i = read_corpus(valid_data_path_i, vocab2idx, label2idx)
    valid_datas.append(valid_datas_i)
    valid_labels.append(valid_labels_i)

In [72]:
# 提取训练集已有规则
train_files = os.listdir('data/train')
train_ann_files = [x for x in train_files if x.endswith('.ann')]
tag_dic = {}
del_list = []
for file in train_ann_files:
    with open('data/train/%s'%file) as file_obj:
        labeled_data = file_obj.read()
    labeled_data_list = labeled_data.split('\n')
    for labeled in labeled_data_list:
        res = re.split(' |\t',labeled)
        tag = res[1]
        word = res[4]
        if word not in tag_dic and word not in del_list:
            tag_dic[word] = tag
        else:
            try:
                del tag_dic[word]
            except:
                pass
            del_list.append(word)

In [73]:
print(train_datas[50])
print([idx2vocab[idx] for idx in train_datas[50]])
print(train_labels[50])
print([idx2label[idx] for idx in train_labels[50]])

[58, 61, 77, 1, 58, 17, 181, 3093, 3817, 2654, 6214, 1959, 2177, 286, 6802, 5965, 519, 1408, 2644, 2102, 2732, 1842, 889, 2545, 3093, 3817]
['_', 'b', 'r', '<UNK>', '_', '3', '、', '治', '疗', '期', '间', '忌', '房', '事', '，', '配', '偶', '如', '有', '感', '染', '应', '同', '时', '治', '疗']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


### CRF+BiLSTM

In [74]:
import numpy as np
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Masking, Embedding, Bidirectional, LSTM, Dense, Input, TimeDistributed, Activation
from keras.preprocessing import sequence
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
from keras import backend as K
K.clear_session()

EPOCHS = 40
BATCH_SIZE = 128
EMBED_DIM = 72
HIDDEN_SIZE = 18
MAX_LEN = 100
VOCAB_SIZE = len(vocab2idx)
CLASS_NUMS = len(label2idx)
print(VOCAB_SIZE, CLASS_NUMS)

print('padding sequences')
train_datas = sequence.pad_sequences(train_datas, maxlen=MAX_LEN)
train_labels = sequence.pad_sequences(train_labels, maxlen=MAX_LEN)
valid_datas = sequence.pad_sequences(valid_datas, maxlen=MAX_LEN)
valid_labels = sequence.pad_sequences(valid_labels, maxlen=MAX_LEN)
print('x_train shape:', train_datas.shape)
print('x_test shape:', valid_datas.shape)

train_labels = keras.utils.to_categorical(train_labels, CLASS_NUMS)
valid_labels = keras.utils.to_categorical(valid_labels, CLASS_NUMS)
print('trainlabels shape:', train_labels.shape)
print('testlabels shape:', valid_labels.shape)

## BiLSTM+CRF模型构建
inputs = Input(shape=(MAX_LEN,), dtype='int32')
x = Masking(mask_value=0)(inputs)
x = Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True)(x)
x = Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True))(x)
x = TimeDistributed(Dense(CLASS_NUMS))(x)#TimeDistributed层的作用就是把Dense层应用到这10个具体的向量上，对每一个向量进行了一个Dense操作
outputs = CRF(CLASS_NUMS)(x)
model = Model(inputs=inputs, outputs=outputs)
model.summary()

model.compile(loss=crf_loss, optimizer='adam', metrics=[crf_viterbi_accuracy])
model.fit(train_datas, train_labels, epochs=EPOCHS, verbose=1, validation_split=0.1)

score = model.evaluate(valid_datas, valid_labels, batch_size=BATCH_SIZE)
print(model.metrics_names)
print(score)

6874 21
padding sequences
x_train shape: (6899, 100)
x_test shape: (3974, 100)
trainlabels shape: (6899, 100, 21)
testlabels shape: (3974, 100, 21)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
masking_1 (Masking)          (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 72)           494928    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 36)           13104     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 100, 21)           777       
_________________________________________________________________
crf_1 (CRF)                  (None, 100, 21)           945  

In [75]:
def get_valid_nertag(input_data, result_tags):
    result_words = []
    start, end =0, 1 # 实体开始结束位置标识
    tag_label = "O" # 实体类型标识
    number = 0
    for i, tag in enumerate(result_tags):
        if tag.startswith("B"):
            number += 1
            if tag_label != "O": # 当前实体tag之前有其他实体     
                result_words.append(('T'+str(number), tag_label, start, end,input_data[start: end]))# 获取实体 
            tag_label = tag.split("-")[1] # 获取当前实体类型
            start, end = i, i+1 # 开始和结束位置变更
        elif tag.startswith("I"):
            temp_label = tag.split("-")[1]
            if temp_label == tag_label: # 当前实体tag是之前实体的一部分
                end += 1 # 结束位置end扩展
        elif tag == "O":
            if tag_label != "O": # 当前位置非实体 但是之前有实体
                result_words.append(('T'+str(number), tag_label, start, end,input_data[start: end]))
                tag_label = "O"  # 实体类型置"O"
            start, end = i, i+1 # 开始和结束位置变更
    if tag_label != "O": # 最后结尾还有实体
        number += 1
        result_words.append(('T'+str(number), tag_label, start, end,input_data[start: end]))# 获取结尾的实体
    return result_words

In [125]:
# 线下验证数据集
maxlen = 100
valid_result = {}
valid_data_path = 'data/valid/text/'
#test_data_path = 'data/valid/'
for i in range(len(os.listdir(valid_data_path))):
    valid_file = valid_data_path+str(i+800)+'.txt'
    with open(valid_file, "r", encoding="utf8") as valid:
        sentence = valid.read()
    sentences = sentence.split('。')
    y_ner = []
    for sent in sentences:
        sent = sent.replace(' ','_')
        sent_chars = list(sent+'。')
        sent2id = [vocab2idx[word] if word in vocab2idx else vocab2idx['<UNK>'] for word in sent_chars]

        sent2id_new = np.array([[0] * (maxlen-len(sent2id)) + sent2id[:maxlen]])
        y_pred = model.predict(sent2id_new)
        y_label = np.argmax(y_pred, axis=2)
        y_label = y_label.reshape(1, -1)[0]
        y_ner_ = [idx2label[i] for i in y_label][-len(sent_chars):]
        y_ner.extend(y_ner_)
    result_words = get_valid_nertag(sentence, y_ner)
    ans = []
    for res in result_words:
        number = res[0]
        #tag = res[1]
        start = res[2]
        end = res[3]
        word = res[4]
        word = re.sub(' |，|：|；|、|;|。|（|）|\(|\)|,','_',res[4]) # 有特殊字符的实体不予加入结果
        try:
            tag = tag_dic[word]
        except:
            tag = res[1]
        if '_' not in word:
            ans.append('{}\t{} {} {}\t{}'.format('T'+str(len(ans)+1), tag,start, end, word))
            print('{}\t{} {} {}\t{}'.format(str(i+800)+'-T'+str(len(ans)), tag,start, end, word))
        #else:
        #    if len(re.findall('_',word))==1 and len(word)>2:
        #        if word.startswith('_'):
        #            ans.append('{}\t{} {} {}\t{}'.format('T'+str(len(ans)+1), tag,start+1, end, word[1:]))
        #            print('{}\t{} {} {}\t{}'.format(str(i+800)+'-T'+str(len(ans)), tag,start+1, end, word[1:]))
        #        if word.endswith('_'):
        #            ans.append('{}\t{} {} {}\t{}'.format('T'+str(len(ans)+1), tag,start, end-1, word[:-1]))
        #            print('{}\t{} {} {}\t{}'.format(str(i+800)+'-T'+str(len(ans)), tag,start+1, end-1, word[:-1]))
    valid_result[i+800] = ans

800-T1	DRUG_DOSAGE 4 8	薄膜衣片
800-T2	DRUG_TASTE 19 21	味涩
800-T3	DRUG_TASTE 22 24	微苦
800-T4	FOOD_GROUP 31 33	辛辣
800-T5	FOOD_GROUP 34 36	生冷
800-T6	FOOD_GROUP 37 39	油腻
800-T7	DISEASE 44 47	糖尿病
800-T8	SYMPTOM 113 117	月经量多
800-T9	SYMPTOM 127 129	尿频
800-T10	SYMPTOM 130 132	尿急
800-T11	SYMPTOM 133 135	尿痛
800-T12	PERSON_GROUP 232 237	过敏体质者
800-T13	PERSON_GROUP 266 268	儿童
800-T14	DRUG_DOSAGE 319 322	薄膜衣
800-T15	PERSON_GROUP 325 327	孕妇
800-T16	FOOD_GROUP 509 511	多生
800-T17	FOOD_GROUP 511 513	冷辛
801-T1	DRUG_DOSAGE 15 18	胶囊剂
801-T2	DRUG_DOSAGE 30 32	粉未
801-T3	DRUG_TASTE 33 35	气香
801-T4	DRUG_TASTE 36 39	味微苦
801-T5	DRUG_EFFICACY 42 46	活血化瘀
801-T6	DRUG_EFFICACY 47 51	软坚散结
801-T7	DISEASE 54 58	子宫肌瘤
801-T8	DISEASE 59 62	盆腔炎
801-T9	DRUG_EFFICACY 154 158	所致的家
801-T10	DRUG_EFFICACY 159 163	血液粘滞
801-T11	SYMPTOM 166 170	制正常大
801-T12	DISEASE 171 174	离体子
801-T13	SYMPTOM 178 182	收缩频率
802-T1	DRUG_DOSAGE 77 79	栓剂
802-T2	FOOD_GROUP 84 86	辛辣
802-T3	PERSON_GROUP 87 89	孕妇
802-T4	DRUG_EFFICACY 124 128	清热解毒
802-T5	DRUG_E

816-T1	DRUG_DOSAGE 27 30	硬胶囊
816-T2	DRUG_DOSAGE 43 45	粉末
816-T3	DRUG_TASTE 46 48	气微
816-T4	DRUG_TASTE 49 51	味苦
816-T5	DRUG_TASTE 51 53	气微
816-T6	DRUG_EFFICACY 236 240	发生药物
816-T7	SYNDROME 248 252	咨询医师
816-T8	DISEASE 276 279	的腰腹
816-T9	FOOD_GROUP 403 405	收缩
816-T10	FOOD_GROUP 408 410	体反
816-T11	SYMPTOM 414 420	提高热板法小
816-T12	SYNDROME 426 430	少冰醋酸
816-T13	SYMPTOM 459 465	胃明显虚弱者
816-T14	PERSON_GROUP 529 531	其他
816-T15	PERSON_GROUP 532 534	病者
816-T16	PERSON_GROUP 535 537	应去
817-T1	DRUG_DOSAGE 12 14	颗粒
817-T2	DRUG_TASTE 15 17	气香
817-T3	DRUG_TASTE 18 20	味甜
817-T4	DRUG_TASTE 21 23	微苦
817-T5	SYMPTOM 85 89	月经过多
817-T6	DRUG_EFFICACY 104 106	镇痛
817-T7	DRUG_EFFICACY 107 109	抗炎
817-T8	DRUG_EFFICACY 110 116	改善血液循环
817-T9	DRUG_EFFICACY 141 145	活血逐瘀
817-T10	DRUG_EFFICACY 146 150	祛寒止痛
817-T11	SYMPTOM 160 164	月经不调
817-T12	SYMPTOM 165 169	小腹胀痛
817-T13	PERSON_GROUP 220 222	孕妇
817-T14	SYMPTOM 236 240	月经不调
817-T15	SYMPTOM 241 245	小腹胀痛
817-T16	SYMPTOM 245 249	月经不调
817-T17	SYMPTOM 250 254	小腹胀痛
818-T1	DRUG_DOS

831-T1	DRUG_DOSAGE 1 4	胶囊剂
831-T2	DRUG_TASTE 67 69	清淡
831-T3	PERSON_GROUP 160 165	过敏体质者
831-T4	PERSON_GROUP 186 188	儿童
831-T5	PERSON_GROUP 207 209	儿童
831-T6	DRUG_EFFICACY 246 250	清热解毒
831-T7	SYMPTOM 253 257	发热口渴
831-T8	SYMPTOM 258 262	咽喉肿痛
831-T9	SYMPTOM 263 267	热疖疮疡
831-T10	SYMPTOM 268 272	小儿胎毒
831-T11	FOOD_GROUP 272 274	清淡
832-T1	DRUG_EFFICACY 65 69	健脾固肾
832-T2	DRUG_EFFICACY 70 74	养血安胎
832-T3	SYNDROME 77 79	肾虚
832-T4	SYNDROME 81 85	气血虚弱
832-T5	SYNDROME 95 98	产味甜
832-T6	DRUG_DOSAGE 178 180	颗粒
832-T7	DRUG_TASTE 181 183	味甜
832-T8	DRUG_TASTE 184 186	微苦
832-T9	SYNDROME 198 200	肾虚
832-T10	SYNDROME 202 206	气血虚弱
833-T1	DRUG_EFFICACY 28 32	补气养血
833-T2	SYNDROME 35 39	气血两虚
833-T3	DRUG_EFFICACY 39 43	补气养血
833-T4	SYNDROME 46 50	气血两虚
833-T5	SYMPTOM 51 55	头晕目眩
833-T6	SYMPTOM 56 60	心悸失眠
833-T7	SYMPTOM 61 65	食欲不振
833-T8	SYMPTOM 66 68	贫血
834-T1	DRUG_EFFICACY 59 63	清热解毒
834-T2	DRUG_EFFICACY 69 73	收敛止痒
834-T3	DISEASE 83 86	阴道炎
834-T4	DRUG_EFFICACY 94 98	清热解毒
834-T5	DRUG_EFFICACY 99 103	燥湿杀虫
834-T6	DRUG_

853-T1	DRUG_EFFICACY 1 5	清热解毒
853-T2	SYNDROME 13 17	湿热下注
853-T3	SYMPTOM 28 32	经期腹痛
853-T4	DISEASE 33 38	慢性盆腔炎
853-T5	DISEASE 39 42	附件炎
853-T6	DISEASE 43 46	附件炎
853-T7	SYNDROME 213 217	湿热下注
853-T8	SYMPTOM 228 232	经期腹痛
853-T9	DISEASE 233 238	慢性盆腔炎
853-T10	DISEASE 239 242	附件炎
853-T11	DRUG_DOSAGE 266 269	胶囊剂
853-T12	DRUG_DOSAGE 282 284	颗粒
853-T13	DRUG_TASTE 285 288	气微香
853-T14	DRUG_TASTE 289 291	味苦
854-T1	DISEASE 60 65	缺铁性贫血
854-T2	DRUG_DOSAGE 123 127	薄膜衣片
854-T3	DRUG_TASTE 143 145	味酸
854-T4	DRUG_TASTE 146 147	甜
854-T5	SYMPTOM 237 239	皮疹
855-T1	DRUG_INGREDIENT 13 16	夏枯草
855-T2	DRUG_EFFICACY 52 54	清火
855-T3	DRUG_EFFICACY 55 57	散结
855-T4	DRUG_EFFICACY 58 60	消肿
855-T5	SYMPTOM 73 75	眩晕
855-T6	SYMPTOM 76 78	瘰疬
855-T7	SYMPTOM 79 81	瘿瘤
855-T8	SYMPTOM 82 86	乳痈肿痛
855-T9	SYMPTOM 87 92	甲状腺肿大
856-T1	PERSON_GROUP 1 3	孕妇
856-T2	DRUG_DOSAGE 58 61	硬胶囊
856-T3	DRUG_DOSAGE 73 75	粉末
856-T4	DRUG_TASTE 76 79	气微香
856-T5	DRUG_TASTE 80 82	味苦
856-T6	DRUG_TASTE 83 85	微咸
856-T7	SYNDROME 109 113	气虚血瘀
856-T8	SYMPTOM 11

871-T1	DRUG_DOSAGE 63 66	硬胶囊
871-T2	DRUG_DOSAGE 82 84	粉末
871-T3	DRUG_TASTE 85 87	气腥
871-T4	DRUG_TASTE 88 90	味苦
871-T5	SYNDROME 95 99	肝郁气滞
871-T6	SYNDROME 100 104	痰瘀凝聚
871-T7	SYMPTOM 112 116	乳房胀痛
871-T8	DRUG_EFFICACY 153 157	疏肝解郁
871-T9	DRUG_EFFICACY 158 162	化痰散结
871-T10	DRUG_EFFICACY 163 167	活血止痛
871-T11	SYNDROME 170 174	肝郁气滞
871-T12	SYNDROME 175 179	痰瘀凝聚
871-T13	SYMPTOM 187 191	乳房胀痛
872-T1	DRUG_EFFICACY 44 48	补气养血
872-T2	DRUG_EFFICACY 67 71	补气养血
872-T3	DRUG_EFFICACY 72 76	调经止带
872-T4	SYNDROME 79 83	气血两虚
872-T5	SYMPTOM 84 88	身体瘦弱
872-T6	SYMPTOM 89 93	腰膝酸软
872-T7	SYMPTOM 94 98	月经量少
872-T8	SYMPTOM 102 104	带下
872-T9	DRUG_EFFICACY 104 108	补气养血
872-T10	DRUG_EFFICACY 109 113	调经止带
872-T11	PERSON_GROUP 113 115	孕妇
872-T12	DRUG_INGREDIENT 166 168	人参
872-T13	DRUG_INGREDIENT 169 171	白芍
872-T14	DRUG_INGREDIENT 172 175	反藜芦
872-T15	DRUG_INGREDIENT 179 181	藜芦
872-T16	DRUG_INGREDIENT 193 195	甘草
872-T17	DRUG_INGREDIENT 196 199	反甘遂
872-T18	DRUG_INGREDIENT 200 202	大戟
872-T19	DRUG_INGREDIENT 203 205	海藻
872

...

927-T1	DRUG_EFFICACY 16 20	调经养血
927-T2	DRUG_EFFICACY 21 25	顺气化瘀
927-T3	SYMPTOM 33 37	赶前错后
927-T4	SYMPTOM 38 42	腰腿酸痛
927-T5	SYMPTOM 43 47	腹痛胀满
927-T6	DRUG_DOSAGE 47 49	片剂
927-T7	DRUG_DOSAGE 50 53	薄膜衣
927-T8	PERSON_GROUP 63 65	孕妇
927-T9	SYMPTOM 84 88	月经不调
927-T10	SYMPTOM 89 93	经期紊乱
927-T11	SYMPTOM 94 98	腰腿酸痛
927-T12	DRUG_DOSAGE 168 170	片剂
927-T13	DRUG_DOSAGE 171 174	薄膜衣
927-T14	FOOD_GROUP 223 225	寒凉
927-T15	FOOD_GROUP 226 228	生冷
927-T16	DISEASE 233 235	感冒
927-T17	FOOD_GROUP 254 256	寒凉
927-T18	FOOD_GROUP 257 259	生冷
927-T19	DISEASE 264 266	感冒
927-T20	SYMPTOM 301 305	月经量少
927-T21	SYMPTOM 307 311	月经错后
927-T22	SYMPTOM 320 324	月经量少
927-T23	SYMPTOM 326 330	月经错后
927-T24	SYMPTOM 332 339	阴道不规则出血
927-T25	PERSON_GROUP 398 403	过敏体质者
927-T26	PERSON_GROUP 430 432	儿童
927-T27	DRUG_TASTE 477 480	气芳香
927-T28	DRUG_TASTE 481 484	味微苦
928-T1	DRUG_EFFICACY 1 5	清热解毒
928-T2	DRUG_EFFICACY 6 10	消肿散结
928-T3	DISEASE 13 16	附件炎
928-T4	DISEASE 17 20	附件炎
928-T5	DISEASE 28 33	慢性盆腔炎
928-T6	DISEASE 34 37	附件炎
928-T7	DISEASE 

945-T1	SYMPTOM 65 68	耳肿胀
945-T2	SYMPTOM 166 169	甲苯致
945-T3	DISEASE 239 242	肿散结
945-T4	DISEASE 243 246	用于附
945-T5	DISEASE_GROUP 285 288	热下注
945-T6	DISEASE 289 294	致的带下病
945-T7	DISEASE 295 298	慢性盆
945-T8	DRUG_TASTE 327 329	显浅
945-T9	PERSON_GROUP 366 368	一疗
946-T1	DRUG_EFFICACY 6 10	祛风通络
946-T2	DRUG_EFFICACY 11 15	舒筋止痛
946-T3	SYMPTOM 22 27	中度神经根
946-T4	DISEASE 28 31	颈椎病
946-T5	SYNDROME 31 33	气虚
946-T6	SYNDROME 33 37	血瘀证出
946-T7	SYMPTOM 56 60	上肢麻木
946-T8	SYMPTOM 61 65	神疲乏力
946-T9	DRUG_DOSAGE 65 69	薄膜衣丸
946-T10	DRUG_DOSAGE 73 77	薄膜衣丸
946-T11	DRUG_TASTE 93 95	气微
946-T12	DRUG_TASTE 96 98	味苦
946-T13	PERSON_GROUP 117 120	运动员
946-T14	SYMPTOM 200 202	腹痛
946-T15	SYMPTOM 206 210	腹部不适
946-T16	SYMPTOM 246 251	性心律不齐
946-T17	PERSON_GROUP 335 337	孕妇
947-T1	DRUG_INGREDIENT 19 21	消丹
947-T2	FOOD_GROUP 42 44	食酸
947-T3	FOOD_GROUP 47 50	刺激性
947-T4	SYNDROME 57 61	气滞血瘀
947-T5	DISEASE_GROUP 63 69	乳腺小叶增生
947-T6	SYMPTOM 70 74	子宫肌瘤
947-T7	DRUG_DOSAGE 84 88	薄膜衣片
947-T8	DRUG_TASTE 103 106	气微香
947-T9	DRUG_TASTE 107 11

965-T1	DRUG_EFFICACY 36 40	滋阴补血
965-T2	DRUG_EFFICACY 41 45	健脾益气
965-T3	FOOD_GROUP 63 65	寒凉
965-T4	FOOD_GROUP 66 68	生冷
965-T5	PERSON_GROUP 72 74	孕妇
965-T6	DISEASE 75 78	糖尿病
965-T7	DISEASE 91 93	感冒
965-T8	SYMPTOM 128 132	月经量少
965-T9	SYMPTOM 134 138	月经错后
965-T10	SYMPTOM 140 147	阴道不规则出血
965-T11	PERSON_GROUP 240 245	过敏体质者
965-T12	PERSON_GROUP 274 276	儿童
965-T13	DRUG_DOSAGE 320 322	颗粒
965-T14	DRUG_TASTE 323 325	味甜
965-T15	DRUG_EFFICACY 337 341	滋阴补血
965-T16	DRUG_EFFICACY 342 346	健脾益气
965-T17	DRUG_EFFICACY 347 351	调经活血
965-T18	SYNDROME 359 363	气虚血亏
965-T19	SYMPTOM 364 368	月经不调
966-T1	SYNDROME 3 7	气虚血瘀
966-T2	SYMPTOM 12 16	恶露不绝
966-T3	SYMPTOM 19 25	产后出血过多
966-T4	SYMPTOM 26 30	淋漓不断
966-T5	SYMPTOM 31 35	神疲乏力
966-T6	SYMPTOM 36 38	腰腿
966-T7	DRUG_EFFICACY 49 53	活血调经
966-T8	DRUG_EFFICACY 54 58	祛瘀止痛
966-T9	DRUG_EFFICACY 73 77	益气养血
966-T10	DRUG_EFFICACY 78 82	补气健脾
966-T11	DRUG_EFFICACY 178 181	组方滋
966-T12	DRUG_EFFICACY 190 194	伐而不伤
966-T13	DRUG_DOSAGE 251 253	减少
966-T14	DRUG_TASTE 254 257	粘膜腺
966-T15	D

983-T1	DRUG_DOSAGE 27 29	液体
983-T2	DRUG_TASTE 30 32	气微
983-T3	DRUG_TASTE 33 35	味甜
983-T4	FOOD_GROUP 96 98	辛辣
983-T5	FOOD_GROUP 98 101	刺激性
983-T6	DRUG_EFFICACY 187 189	安胎
983-T7	DRUG_EFFICACY 216 218	保胎
983-T8	DRUG_EFFICACY 265 269	健脾固肾
983-T9	DRUG_EFFICACY 270 274	养血安胎
983-T10	SYNDROME 277 279	肾虚
983-T11	SYNDROME 281 285	气血虚弱
983-T12	DRUG_EFFICACY 312 318	固肾养血安胎
983-T13	SYNDROME 321 323	肾虚
983-T14	SYNDROME 325 329	气血虚弱
984-T1	SYMPTOM 23 27	月经不调
984-T2	SYMPTOM 36 42	产后瘀血腹痛
984-T3	SYMPTOM 73 77	月经不调
984-T4	SYMPTOM 86 92	产后瘀血腹痛
984-T5	DRUG_DOSAGE 114 117	胶囊剂
984-T6	DRUG_TASTE 128 130	气微
984-T7	DRUG_TASTE 131 133	味咸
984-T8	DRUG_TASTE 134 136	微苦
984-T9	PERSON_GROUP 141 143	孕妇
984-T10	SYNDROME 148 152	身体虚弱
984-T11	SYMPTOM 153 157	脾肾虚寒
985-T1	PERSON_GROUP 0 2	孕妇
985-T2	PERSON_GROUP 41 43	孕妇
985-T3	DRUG 71 74	赤石脂
985-T4	DRUG_INGREDIENT 85 87	肉桂
985-T5	SYMPTOM 98 102	口干舌燥
985-T6	DISEASE 115 118	糖尿病
985-T7	DISEASE 119 123	慢性肾炎
985-T8	DISEASE 124 127	高血压
985-T9	DISEASE 128 131	心脏病
985-T10	PERSON_

997-T1	DRUG_DOSAGE 22 24	丸剂
997-T2	DRUG_DOSAGE 25 28	水蜜丸
997-T3	SYMPTOM 33 37	子宫寒冷
997-T4	SYMPTOM 38 42	月经量少
997-T5	SYMPTOM 46 48	痛经
997-T6	DRUG_DOSAGE 103 106	水蜜丸
997-T7	DRUG_TASTE 107 109	味苦
997-T8	DRUG_TASTE 110 111	甜
997-T9	DRUG_TASTE 112 114	略涩
997-T10	FOOD_GROUP 121 123	生冷
997-T11	DISEASE 134 136	感冒
997-T12	SYMPTOM 179 183	月经过少
997-T13	SYMPTOM 185 189	经期错后
997-T14	SYMPTOM 191 198	阴道不规则出血
997-T15	SYMPTOM 286 291	痛经不减轻
997-T16	PERSON_GROUP 392 394	儿童
997-T17	DRUG_EFFICACY 436 440	补气养血
997-T18	SYNDROME 448 452	气血凝滞
997-T19	SYMPTOM 453 457	子宫寒冷
997-T20	SYMPTOM 458 462	月经量少
997-T21	SYMPTOM 466 468	痛经
997-T22	SYMPTOM 469 473	白带量多
997-T23	SYMPTOM 474 478	小腹下坠
997-T24	PERSON_GROUP 504 506	孕妇
997-T25	DISEASE 509 512	糖尿病
998-T1	SYMPTOM 19 23	恶露不行
998-T2	SYMPTOM 24 28	少腹疼痛
998-T3	SYMPTOM 37 41	阴道流血
998-T4	SYMPTOM 42 46	月经过多
998-T5	DRUG_EFFICACY 83 85	活血
998-T6	DRUG_EFFICACY 86 88	祛瘀
998-T7	DRUG_EFFICACY 89 91	止痛
998-T8	SYMPTOM 96 100	恶露不行
998-T9	SYMPTOM 101 105	少腹疼痛
998-T10	SYMPTOM 120 124	

In [126]:
# 预测验证集
valid_pred = []
for i in range(800,1000):
    valid_pred.extend(list(map(lambda x:str(i)+' '+re.sub(r'T[\d+]{1,2}\t','',x),valid_result[i])))
    
    
    

In [127]:
# 验证集的真实label
valid_true = []
for i in range(800,1000):
    with open('data/valid/ann/%d.ann'%i) as file_obj:
        valid_label = file_obj.read().split('\n')
        valid_true.extend(list(map(lambda x:str(i)+' '+re.sub(r'T[\d+]{1,2}\t','',x),valid_label)))


In [128]:
def get_len(text):
    text = valid_pred[1]
    text_list = re.split(' |\t',text)
    return len(text_list[-1])

In [129]:
# strict F1
def eval(valid_pred,valid_true):
    beta = 1
    #P = len(set(valid_pred) & set(valid_true))/len(set(valid_true))
    #R = len(set(valid_pred) & set(valid_true))/len(set(valid_pred))
    X,Y,Z = 0,0,0
    for text in valid_pred:
        n = get_len(text)
        if text in valid_true:
            X+=n
        Y+=n

    for text in valid_true:
        n = get_len(text)
        Z+=n
    P = X/Z
    R = X/Y
    F = (1+beta**2)*P*R/(P+beta**2*R)
    print('P = %.4f'%P)
    print('R = %.4f'%R)
    print('F = %.4f'%F)
    return P,R,F

In [130]:
# strict F1
#def eval(valid_pred,valid_true):
#    beta = 1
#    P = len(set(valid_pred) & set(valid_true))/len(set(valid_true))
#    R = len(set(valid_pred) & set(valid_true))/len(set(valid_pred))
#    F = (1+beta**2)*P*R/(P+beta**2*R)
#    print('P = %.4f'%P)
#    print('R = %.4f'%R)
#    print('F = %.4f'%F)
#    return P,R,F

In [131]:
results = eval(valid_pred,valid_true)
results

P = 0.4686
R = 0.4956
F = 0.4817


(0.4685796269727403, 0.4955993930197269, 0.48171091445427733)

In [132]:
para = {}
para['EPOCHS'] = EPOCHS
para['BATCH_SIZE'] = BATCH_SIZE
para['EMBED_DIM'] = EMBED_DIM
para['HIDDEN_SIZE'] = HIDDEN_SIZE
para['MAX_LEN'] = MAX_LEN
para['VOCAB_SIZE'] = VOCAB_SIZE
para['CLASS_NUMS'] = CLASS_NUMS
para['VALID_PRED'] = len(set(valid_pred))
para['VALID_TRUE'] = len(set(valid_true))
para['P'] = results[0]
para['R'] = results[1]
para['F'] = results[2]

log = []
for p in para:
    log.append(p+' = '+str(para[p]))
    

import datetime
time = datetime.datetime.now().strftime('%Y%m%d%H%M')

with open('model/log/log_%s.txt'%time,'w', encoding='utf-8') as wr:
    wr.write('\n'.join(log))
# save model
model.save("model/model/model_%s.h5"%time)   

In [133]:
# output
result = {}
test_data_path = 'data/chusai_xuanshou/'
for i in range(len(os.listdir(test_data_path))):
    test_file = test_data_path+str(i+1000)+'.txt'
    with open(test_file, "r", encoding="utf8") as test:
        sentence = test.read()
    sentences = sentence.split('。')
    y_ner = []

    for sent in sentences:
        sent = sent.replace(' ','_')
        sent_chars = list(sent+'。')
        sent2id = [vocab2idx[word] if word in vocab2idx else vocab2idx['<UNK>'] for word in sent_chars]

        sent2id_new = np.array([[0] * (maxlen-len(sent2id)) + sent2id[:maxlen]])
        y_pred = model.predict(sent2id_new)
        y_label = np.argmax(y_pred, axis=2)
        y_label = y_label.reshape(1, -1)[0]
        y_ner_ = [idx2label[i] for i in y_label][-len(sent_chars):]
        y_ner.extend(y_ner_)
    result_words = get_valid_nertag(sentence, y_ner)
    ans = []
    for res in result_words:
        number = res[0]
        #tag = res[1]
        start = res[2]
        end = res[3]
        word = re.sub(' |，|：|；|、|;|。|（|）|\(|\)|\d+','_',res[4]) # 有特殊字符的实体不予加入结果
        try:
            tag = tag_dic[word]
        except:
            tag = res[1]
        #if word.startswith('_'):
        #    ans.append('{}\t{} {} {}\t{}'.format('T'+str(len(ans)+1), tag,start+1, end, word[1:]))
        #    print('{}\t{} {} {}\t{}'.format('T'+str(len(ans)), tag,start+1, end, word[1:]))
        #if word.endswith('_'):
        #    ans.append('{}\t{} {} {}\t{}'.format('T'+str(len(ans)+1), tag,start, end-1, word[:-1]))
        #    print('{}\t{} {} {}\t{}'.format('T'+str(len(ans)), tag,start+1, end-1, word[:-1]))
        if '_' not in word:
            ans.append('{}\t{} {} {}\t{}'.format('T'+str(len(ans)+1), tag,start, end, word))       
            print('{}\t{} {} {}\t{}'.format('T'+str(len(ans)), tag,start, end, word))
    
    result[i+1000] = ans

T1	DRUG_EFFICACY 138 142	清热解毒
T2	DRUG_EFFICACY 143 147	化湿除带
T3	DRUG_EFFICACY 148 152	祛瘀止痛
T4	DRUG_EFFICACY 153 156	散结消
T5	DISEASE 160 165	慢性盆腔炎
T6	SYMPTOM 167 171	小腹疼痛
T7	SYMPTOM 172 176	腰骶酸痛
T8	SYMPTOM 177 181	带下量多
T1	PERSON_GROUP 31 33	孕妇
T2	DISEASE 36 39	糖尿病
T3	DRUG_EFFICACY 76 78	补血
T4	DRUG_EFFICACY 79 81	活血
T5	DRUG_EFFICACY 82 84	通络
T6	SYMPTOM 87 91	月经量少
T7	SYMPTOM 95 101	血虚萎黄后错
T8	SYNDROME 102 104	血虚
T9	SYMPTOM 107 111	风湿痹痛
T10	DISEASE 116 119	糖尿病
T1	DISEASE 36 38	感冒
T2	SYMPTOM 38 40	发热
T3	DISEASE 51 54	高血压
T4	DISEASE_GROUP 55 58	心脏病
T5	DISEASE_GROUP 59 61	肝病
T6	DISEASE 62 65	糖尿病
T7	DISEASE_GROUP 66 68	肾病
T8	DISEASE_GROUP 69 72	慢性病
T9	SYMPTOM 90 94	月经紊乱
T10	SYMPTOM 109 111	眩晕
T11	PERSON_GROUP 158 163	过敏体质者
T12	PERSON_GROUP 192 194	儿童
T13	DRUG_DOSAGE 242 244	颗粒
T14	DRUG_TASTE 245 248	气微香
T15	DRUG_TASTE 249 252	味微苦
T16	DRUG_EFFICACY 255 259	滋养肝肾
T17	DRUG_EFFICACY 260 264	宁心安神
T18	SYNDROME 274 276	阴虚
T19	DISEASE_GROUP 276 279	肝旺症
T20	SYMPTOM 282 286	烘热汗出
T21	SYMPTOM 287 291	头晕耳鸣
T22

T1	PERSON_GROUP 8 10	孕妇
T2	DRUG_DOSAGE 22 24	颗粒
T3	DRUG_TASTE 25 27	气微
T4	DRUG_TASTE 28 30	味甜
T5	DRUG_TASTE 31 33	微涩
T6	DRUG_EFFICACY 40 43	清湿热
T7	DRUG_EFFICACY 44 47	止带下
T8	DRUG_EFFICACY 60 65	止带下头晕
T9	PERSON_GROUP 100 102	孕妇
T10	SYMPTOM 126 128	头晕
T1	DRUG_DOSAGE 24 27	水蜜丸
T2	DRUG_DOSAGE 74 77	大蜜丸
T3	DRUG_TASTE 78 80	味甘
T4	DRUG_TASTE 81 84	苦味甘
T5	PERSON_GROUP 89 91	孕妇
T6	DISEASE 94 98	风寒感冒
T7	FOOD_GROUP 111 113	生冷
T8	SYMPTOM 131 135	月经不调
T9	SYMPTOM 136 144	经期不准行经腹痛
T10	DRUG_EFFICACY 151 155	舒郁化滞
T11	SYNDROME 158 162	气虚血寒
T12	SYNDROME 163 167	肝郁不舒
T13	SYMPTOM 170 174	经期不准
T14	SYMPTOM 175 179	行经腹痛
T15	SYMPTOM 180 184	赤白带下
T16	SYMPTOM 185 189	两胁胀痛
T17	SYMPTOM 190 194	倦怠食少
T1	FOOD_GROUP 5 7	寒凉
T2	FOOD_GROUP 8 10	生冷
T3	PERSON_GROUP 15 17	孕妇
T4	DISEASE 18 21	糖尿病
T5	DISEASE 35 37	感冒
T6	SYMPTOM 86 90	月经量少
T7	SYMPTOM 92 96	月经错后
T8	SYMPTOM 98 105	阴道不规则出血
T9	PERSON_GROUP 164 169	过敏体质者
T10	PERSON_GROUP 199 201	儿童
T11	DRUG_EFFICACY 284 288	补血活血
T12	SYNDROME 296 298	血虚
T13	SYMPTOM 299 303	月经不调
T14	

T1	DRUG_EFFICACY 1 5	健脾和胃
T2	DRUG_EFFICACY 6 10	养血安神
T3	SYMPTOM 13 17	脾胃虚弱
T4	SYNDROME 18 22	心脾两虚
T5	SYNDROME 25 27	血虚
T6	SYMPTOM 31 35	面色萎黄
T7	SYMPTOM 36 39	白光白
T8	SYMPTOM 40 42	食少
T9	SYMPTOM 45 49	脘腹胀闷
T10	SYMPTOM 50 54	大便不调
T11	SYMPTOM 60 64	倦怠乏力
T12	SYMPTOM 65 69	舌胖色淡
T13	SYMPTOM 70 73	苔薄白
T14	SYMPTOM 74 77	脉细弱
T15	DISEASE 78 81	缺铁性
T16	SYMPTOM 81 83	贫血
T17	SYMPTOM 83 85	养血
T18	PERSON_GROUP 97 99	成人
T19	SYMPTOM 344 346	呕吐
T20	SYMPTOM 346 348	恶心
T1	DRUG_EFFICACY 104 108	活血调经
T2	DRUG_EFFICACY 109 113	去瘀止痛
T3	SYMPTOM 118 122	瘀血不净
T4	SYMPTOM 132 136	瘀血不净
T5	DRUG_DOSAGE 145 148	胶囊剂
T6	DRUG_DOSAGE 156 158	颗粒
T7	DRUG_DOSAGE 159 161	粉末
T8	DRUG_TASTE 162 164	气香
T9	SYMPTOM 218 222	<td>
T10	DRUG_EFFICACY 227 231	去瘀止痛
T11	DRUG_EFFICACY 236 240	瘀血不净
T12	SYMPTOM 245 249	去瘀止痛
T1	DRUG_EFFICACY 4 8	补血健脑
T2	SYMPTOM 21 23	贫血
T3	DRUG_EFFICACY 43 46	补气血
T4	SYMPTOM 58 60	头晕
T5	SYMPTOM 61 63	心悸
T1	DRUG_DOSAGE 4 8	薄膜衣片
T2	DRUG_TASTE 23 26	气微香
T3	SYNDROME 27 29	味微
T4	DRUG_DOSAGE 48 51	薄膜衣
T5	PERSON_GROUP 1

T1	PERSON_GROUP 16 18	孕妇
T2	DRUG_INGREDIENT 87 89	香附
T3	DRUG_INGREDIENT 90 92	延胡
T4	DRUG_INGREDIENT 93 95	赤芍
T5	DRUG_EFFICACY 96 102	抑制子宫收缩
T6	SYMPTOM 130 134	月经不调
T7	SYMPTOM 135 137	痛经
T8	DRUG_EFFICACY 147 151	养血舒肝
T9	DRUG_EFFICACY 152 156	调经止痛
T10	SYNDROME 159 161	血虚
T11	SYNDROME 161 163	肝郁
T12	SYMPTOM 165 169	月经不调
T13	SYMPTOM 170 172	痛经
T14	SYMPTOM 182 186	行经后错
T15	SYMPTOM 187 191	经水量少
T16	SYMPTOM 192 195	有血块
T17	SYMPTOM 198 202	小腹疼痛
T18	PERSON_GROUP 320 322	孕妇
T19	SYNDROME 329 331	血虚
T20	SYNDROME 331 333	肝郁
T21	SYMPTOM 335 339	月经不调
T22	SYMPTOM 340 342	痛经
T1	DRUG_DOSAGE 99 101	丸剂
T2	DRUG_DOSAGE 102 104	水丸
T3	FOOD_GROUP 110 112	辛辣
T4	FOOD_GROUP 113 115	生冷
T5	FOOD_GROUP 116 118	油腻
T6	DISEASE 124 127	高血压
T7	DISEASE_GROUP 128 131	心脏病
T8	DISEASE_GROUP 132 134	肝病
T9	DISEASE 135 138	糖尿病
T10	DISEASE_GROUP 139 141	肾病
T11	DISEASE_GROUP 142 145	慢性病
T12	PERSON_GROUP 163 167	孕妇少女
T13	PERSON_GROUP 168 170	孕妇
T14	PERSON_GROUP 232 237	过敏体质者
T15	PERSON_GROUP 264 266	儿童
T16	DRUG_DOSAGE 363 365	水丸
T17

T1	DRUG_DOSAGE 12 15	糖衣片
T2	DRUG_DOSAGE 16 20	薄膜衣片
T3	PERSON_GROUP 59 61	孕妇
T4	DRUG_EFFICACY 72 74	止痛
T5	DRUG_EFFICACY 87 93	养血调经止痛
T6	SYMPTOM 96 100	月经不调
T1	PERSON_GROUP 36 38	孕妇
T2	DRUG_EFFICACY 72 76	祛瘀生新
T3	PERSON_GROUP 140 142	孕妇
T4	FOOD_GROUP 150 152	生冷
T5	SYNDROME 158 162	气血两虚
T6	SYMPTOM 165 169	月经量少
T7	SYMPTOM 177 181	头晕心悸
T8	SYMPTOM 182 186	疲乏无力
T9	DISEASE 198 201	高血压
T10	DISEASE_GROUP 202 205	心脏病
T11	DISEASE_GROUP 206 208	肾病
T12	DISEASE 209 212	糖尿病
T13	PERSON_GROUP 264 269	青春期少女
T14	PERSON_GROUP 270 275	更年期妇女
T15	SYMPTOM 293 295	腹痛
T16	SYMPTOM 297 301	阴道出血
T17	PERSON_GROUP 378 380	儿童
T18	DRUG_DOSAGE 442 444	颗粒
T19	DRUG_TASTE 445 447	味甜
T20	DRUG_TASTE 448 450	微苦
T21	DRUG_EFFICACY 453 457	活血调经
T22	SYMPTOM 460 464	月经量少
T1	PERSON_GROUP 64 66	孕妇
T2	PERSON_GROUP 78 80	孕妇
T3	DRUG_EFFICACY 89 93	散寒止痛
T4	SYNDROME 98 100	血瘀
T5	SYMPTOM 103 107	月经后期
T6	SYMPTOM 108 110	痛经
T7	SYMPTOM 113 117	行经后错
T8	SYMPTOM 120 124	小腹冷痛
T9	SYMPTOM 125 129	经血紫暗
T1	DRUG_EFFICACY 3 7	收缩子宫
T2	DRUG_EFFICACY 42 

...

T1	SYNDROME 3 5	血虚
T2	SYNDROME 5 7	气滞
T3	SYNDROME 8 12	下焦虚寒
T4	SYMPTOM 15 19	月经不调
T5	SYMPTOM 20 22	痛经
T6	FOOD_GROUP 95 97	寒凉
T7	FOOD_GROUP 98 100	生冷
T8	DISEASE 105 107	感冒
T9	SYMPTOM 126 130	月经量少
T10	SYMPTOM 132 136	月经错后
T11	SYMPTOM 138 145	阴道不规则出血
T12	PERSON_GROUP 204 209	过敏体质者
T13	PERSON_GROUP 236 238	儿童
T14	DRUG_DOSAGE 321 324	水蜜丸
T15	DRUG_TASTE 325 327	气微
T16	DRUG_TASTE 328 333	味甘而后苦
T17	DRUG_EFFICACY 358 360	镇痛
T18	SYMPTOM 363 370	子宫平滑肌收缩
T19	DRUG_INGREDIENT 387 389	艾叶
T20	DRUG_EFFICACY 391 395	抑制纤溶
T21	DRUG_EFFICACY 396 398	抗炎
T22	DRUG_EFFICACY 412 414	镇痛
T23	DRUG_EFFICACY 423 429	抑制子宫收缩
T24	SYMPTOM 477 481	月经不调
T25	SYMPTOM 482 484	痛经
T26	SYMPTOM 485 489	宫寒不孕
T27	SYMPTOM 490 492	崩漏
T28	SYMPTOM 493 495	带下
T29	SYNDROME 499 502	虚寒证
T30	SYMPTOM 512 519	降低肠管紧张性
T31	SYMPTOM 534 536	胃痛
T32	SYMPTOM 537 541	慢性肠炎
T33	SYMPTOM 542 544	尿频
T34	DRUG_EFFICACY 561 565	暖宫调经
T35	SYMPTOM 568 572	子宫虚寒
T36	SYMPTOM 573 577	月经不调
T37	SYMPTOM 578 582	经来腹痛
T38	SYMPTOM 583 587	腰酸带下
T39	PERSON_GROUP 587 592	过

T1	PERSON_GROUP 34 36	孕妇
T2	PERSON_GROUP 41 43	孕妇
T3	DRUG_EFFICACY 48 52	祛瘀生新
T4	SYMPTOM 55 59	月经量少
T5	DRUG_EFFICACY 59 63	祛瘀生新
T6	SYMPTOM 66 70	月经量少
T7	SYMPTOM 74 78	经来腹痛
T8	DRUG_EFFICACY 93 97	活血调经
T9	SYNDROME 100 102	血瘀
T10	SYMPTOM 105 109	月经不调
T11	PERSON_GROUP 109 111	孕妇
T1	SYMPTOM 52 56	月经失调
T2	DRUG_EFFICACY 58 62	通调气血
T3	DRUG_EFFICACY 63 67	止痛调经
T4	SYMPTOM 70 74	经期腹痛
T5	SYMPTOM 80 86	月经失调止痛
T6	DRUG_EFFICACY 86 91	调经调气血
T1	DRUG_EFFICACY 46 50	活血调经
T2	SYNDROME 53 57	气血两虚
T3	SYNDROME 59 61	血瘀
T4	SYMPTOM 71 77	月经周期错后
T5	SYMPTOM 78 82	行经量少
T6	SYMPTOM 83 87	精神不振
T7	SYMPTOM 88 92	肢体乏力
T8	SYNDROME 110 114	气血两虚
T9	SYMPTOM 117 121	月经不调
T10	SYMPTOM 124 130	月经周期错后
T11	SYMPTOM 131 135	行经量少
T12	SYMPTOM 136 140	精神不振
T13	SYMPTOM 141 145	肢体乏力
T14	DRUG_EFFICACY 199 202	补气血
T15	DRUG_EFFICACY 203 206	调月经
T1	SYMPTOM 65 69	月经量少
T2	PERSON_GROUP 79 81	孕妇
T3	PERSON_GROUP 117 119	孕妇
T4	DRUG_EFFICACY 124 128	活血调经
T5	SYNDROME 131 133	血瘀
T6	SYMPTOM 136 142	月经不调血瘀
T7	DRUG_EFFICACY 142 146	活血调经
T8	SYNDROME 149

T1	SYNDROME 10 14	气血两虚
T2	SYMPTOM 17 21	月经不调
T3	SYMPTOM 24 30	月经周期错后
T4	SYMPTOM 31 35	行经量少
T5	SYMPTOM 36 40	精神不振
T6	SYMPTOM 41 45	肢体乏力
T7	DRUG_EFFICACY 89 92	补气血
T8	DRUG_EFFICACY 93 96	调月经
T9	DRUG_EFFICACY 132 136	益气养血
T10	DRUG_EFFICACY 137 141	活血调经
T11	SYNDROME 144 148	气血两虚
T12	SYNDROME 150 152	血瘀
T13	SYMPTOM 162 168	月经周期错后
T14	SYMPTOM 169 173	行经量少
T15	SYMPTOM 174 178	精神不振
T16	SYMPTOM 179 183	肢体乏力
T17	DRUG_EFFICACY 183 187	益气养血
T18	DRUG_EFFICACY 188 192	活血调经
T19	DRUG_DOSAGE 192 194	丸剂
T20	DRUG_DOSAGE 195 198	大蜜丸
T21	DRUG_EFFICACY 198 201	补气血
T22	DRUG_DOSAGE 221 223	丸剂
T23	DRUG_DOSAGE 224 227	大蜜丸
T1	PERSON_GROUP 14 16	孕妇
T2	DRUG_EFFICACY 34 38	活血调经
T3	SYNDROME 41 43	血瘀
T4	SYNDROME 50 52	血瘀
T5	SYMPTOM 55 59	月经不调
T6	SYMPTOM 62 66	经水量少
T7	PERSON_GROUP 66 68	孕妇
T8	DRUG_EFFICACY 99 103	祛瘀生新
T9	PERSON_GROUP 146 148	孕妇
T10	PERSON_GROUP 212 214	孕妇
T11	DRUG_EFFICACY 219 223	活血调经
T1	PERSON_GROUP 55 57	孕妇
T2	PERSON_GROUP 61 63	孕妇
T3	DRUG_EFFICACY 88 90	镇痛
T4	SYMPTOM 93 100	子宫平滑肌收缩
T5	DRUG_INGREDI

T1	DRUG_EFFICACY 56 58	止血
T2	DRUG_EFFICACY 166 168	保肝
T3	DRUG_EFFICACY 238 242	增强免疫
T4	DRUG_EFFICACY 489 493	补气养血
T5	DRUG_INGREDIENT 566 568	人参
T6	DRUG_INGREDIENT 569 571	白芍
T7	DRUG_INGREDIENT 572 575	反藜芦
T8	DRUG_INGREDIENT 579 581	藜芦
T9	DRUG_INGREDIENT 593 595	甘草
T10	DRUG_INGREDIENT 596 599	反甘遂
T11	DRUG_INGREDIENT 600 602	大戟
T12	DRUG_INGREDIENT 603 605	海藻
T13	DRUG_INGREDIENT 606 608	芫花
T14	DRUG_INGREDIENT 612 614	甘遂
T15	DRUG_INGREDIENT 615 617	大戟
T16	DRUG_INGREDIENT 618 620	海藻
T17	DRUG_INGREDIENT 621 623	芫花
T18	FOOD_GROUP 636 638	生冷
T19	FOOD_GROUP 639 641	辛辣
T20	FOOD_GROUP 642 644	荤腥
T21	FOOD_GROUP 644 646	油腻
T22	DRUG_EFFICACY 741 747	促进造血生冷
T23	FOOD_GROUP 748 750	辛辣
T24	FOOD_GROUP 751 753	荤腥
T25	FOOD_GROUP 753 755	油腻
T26	DRUG_EFFICACY 848 852	补气养血
T27	SYNDROME 860 864	气血两虚
T28	SYMPTOM 865 869	身体瘦弱
T29	SYMPTOM 870 874	腰膝酸软
T30	SYMPTOM 875 879	月经量少
T31	SYMPTOM 883 885	带下
T32	DRUG_EFFICACY 885 889	补气养血
T33	PERSON_GROUP 897 899	孕妇
T34	PERSON_GROUP 903 905	孕妇
T1	DRUG_EFFICACY 6 10	活血调经
T2

T1	SYNDROME 17 21	气血两虚
T2	SYNDROME 23 25	血瘀
T3	SYMPTOM 35 41	月经周期错后
T4	SYMPTOM 42 46	行经量少
T5	SYMPTOM 47 51	精神不振
T6	SYMPTOM 52 56	肢体乏力
T7	SYMPTOM 56 60	行经量少
T8	SYNDROME 124 128	气血两虚
T9	SYMPTOM 131 135	月经不调
T10	SYMPTOM 138 144	月经周期错后
T11	SYMPTOM 145 149	行经量少
T12	SYMPTOM 155 159	肢体乏力
T13	DRUG_EFFICACY 196 199	补气血
T14	DRUG_EFFICACY 200 203	调月经
T1	DRUG_EFFICACY 8 12	活血化瘀
T2	DRUG_EFFICACY 13 17	缓消瘀块
T3	SYMPTOM 22 26	宿有血块
T4	SYMPTOM 30 34	漏下不止
T5	SYMPTOM 35 39	胎动不安
T6	SYMPTOM 46 50	行经腹痛
T7	SYMPTOM 65 69	腹痛拒按
T8	PERSON_GROUP 70 72	孕妇
T9	SYMPTOM 143 147	行经腹痛
T10	SYMPTOM 150 154	恶露不尽
T1	DRUG_EFFICACY 17 21	调经养血
T2	DRUG_EFFICACY 22 26	理气止痛
T3	SYNDROME 34 36	气滞
T4	SYNDROME 36 38	血瘀
T5	SYMPTOM 41 45	月经不调
T6	SYMPTOM 46 48	痛经
T7	SYMPTOM 49 53	小腹胀痛
T8	FOOD_GROUP 68 70	辛辣
T9	FOOD_GROUP 71 73	生冷
T10	DISEASE 79 81	感冒
T11	SYMPTOM 81 83	发热
T12	DISEASE 94 97	高血压
T13	DISEASE_GROUP 98 101	心脏病
T14	DISEASE_GROUP 102 104	肝病
T15	DISEASE 105 108	糖尿病
T16	DISEASE_GROUP 109 111	肾病
T17	DISEASE_GROUP 112 115	慢性病
T18	PE

In [ ]:
#“实体类别”、“起始位置”、“结束位置”以空格分隔

In [134]:
for i in range(1000,1500):
    with open('data/submit/%d.ann'%i,'w', encoding='utf-8') as wr:
        wr.write('\n'.join(result[i]))